<a href="https://colab.research.google.com/github/Rosireddy-V/Parts_of_speech_nlp/blob/main/POS_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
import string
from collections import defaultdict

In [135]:
with open("WSJ_02-21.pos",'r') as f:
  lines=f.readlines()

In [136]:
print("\t\tWord", "\tTag\n")
for i in range(5):
  print(f"line number {i+1}: {lines[i]}")


		Word 	Tag

line number 1: In	IN

line number 2: an	DT

line number 3: Oct.	NNP

line number 4: 19	CD

line number 5: review	NN



In [137]:
lines[0]

'In\tIN\n'

In [138]:
words=[line.split("\t")[0] for line in lines]
words[:5]

['In', 'an', 'Oct.', '19', 'review']

In [139]:
freq=defaultdict(int)
for word in words:
  freq[word]+=1
tot=freq.items()
print(freq['In'])
print(type(tot))
print(len(tot))

1740
<class 'dict_items'>
44390


In [140]:
voc=[k for k,v in freq.items() if v>1 and k!="\n"]
print(len(voc))

23767


In [141]:
voc.sort()

for i in range(4000,4005):
  print(voc[i])

Early
Earnings
Earth
Earthquake
East


In [142]:
def unkn_word(word):
  punc=set(string.punctuation)
  noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
  verb_suffix = ["ate", "ify", "ise", "ize"]
  adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
  adv_suffix = ["ward", "wards", "wise"]
  if any(char.isdigit() for char in word):
    return "--unk_digit--"
  elif any(char in punc for char in word):
    return "--unk_punct--"
  elif any(char.isupper() for char in word):
    return "--unk_upper--"
  elif any(word.endswith(suffix) for suffix in noun_suffix):
    return "--unk_noun--"
  elif any(word.endswith(suffix) for suffix in verb_suffix):
    return "--unk_verb--"
  elif any(word.endswith(suffix) for suffix in adj_suffix):
    return "--unk_adj--"
  elif any(word.endswith(suffix) for suffix in adv_suffix):
    return "--unk_adv--"

  return "--unk--"

In [143]:
def get_word_tag(line,vocab):
  if not line.split():
    word="--n--"
    tag="--s--"
    return word,tag
  else:
    word,tag=line.split()
    if word not in vocab:
      tag=unkn_word(word)
    return word,tag

  return None

In [144]:
get_word_tag('\n',voc)

('--n--', '--s--')

In [145]:
t='In\tIN\n'
a='\n'
w,ta=t.split()
#wo,tg=a.split()
print(w)
print(ta)


In
IN


In [146]:
get_word_tag('In\tIN\n',voc)

('In', 'IN')

In [147]:
get_word_tag('tardigrade\tNN\n',voc)

('tardigrade', '--unk--')

In [148]:
def preprocess(vocab, data_fp):
    """
    Preprocess data
    """
    orig = []
    prep = []

    # Read data
    with open(data_fp, "r") as data_file:

        for cnt, word in enumerate(data_file):

            # End of sentence
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Handle unknown words
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = unkn_word(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep

In [149]:
import numpy as np
import pandas as pd

In [150]:
tags=['RB','NN','TO']

In [151]:
transition_counts = {
    ('NN', 'NN'): 16241,
    ('RB', 'RB'): 2263,
    ('TO', 'TO'): 2,
    ('NN', 'TO'): 5256,
    ('RB', 'TO'): 855,
    ('TO', 'NN'): 734,
    ('NN', 'RB'): 2431,
    ('RB', 'NN'): 358,
    ('TO', 'RB'): 200
}

In [152]:
num=len(tags)
transition_matrix=np.zeros((num,num))
transition_matrix

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [153]:
sorted_tags=sorted(tags)
sorted_tags

['NN', 'RB', 'TO']

In [154]:
for i in range(num):
  for j in range(num):
    tag_tuple=(sorted_tags[i],sorted_tags[j])
    transition_matrix[i,j]=transition_counts[tag_tuple]

transition_matrix

array([[1.6241e+04, 2.4310e+03, 5.2560e+03],
       [3.5800e+02, 2.2630e+03, 8.5500e+02],
       [7.3400e+02, 2.0000e+02, 2.0000e+00]])

In [155]:
def print_matrix(matrix):
    print(pd.DataFrame(matrix, index=sorted_tags, columns=sorted_tags))

In [156]:
print_matrix(transition_matrix)

         NN      RB      TO
NN  16241.0  2431.0  5256.0
RB    358.0  2263.0   855.0
TO    734.0   200.0     2.0


In [157]:
transition_matrix=transition_matrix/10
print_matrix(transition_matrix)

        NN     RB     TO
NN  1624.1  243.1  525.6
RB    35.8  226.3   85.5
TO    73.4   20.0    0.2


In [158]:
rows_sum=transition_matrix.sum(axis=1,keepdims=True)
rows_sum

array([[2392.8],
       [ 347.6],
       [  93.6]])

In [159]:
transition_matrix=transition_matrix/rows_sum
print_matrix(transition_matrix)

          NN        RB        TO
NN  0.678745  0.101596  0.219659
RB  0.102992  0.651036  0.245972
TO  0.784188  0.213675  0.002137


In [160]:
transition_matrix.sum(axis=1,keepdims=True)

array([[1.],
       [1.],
       [1.]])

In [161]:
t_matrix_for = np.copy(transition_matrix)

In [162]:
import math
for i in range(num):
  t_matrix_for[i,i]+=math.log(rows_sum[i])

In [163]:
print_matrix(t_matrix_for)

          NN        RB        TO
NN  8.458964  0.101596  0.219659
RB  0.102992  6.502088  0.245972
TO  0.784188  0.213675  4.541167


In [164]:
with open("WSJ_02-21.pos",'r') as f:
  training_data=f.readlines()
print(training_data[:5])
print(len(training_data))

['In\tIN\n', 'an\tDT\n', 'Oct.\tNNP\n', '19\tCD\n', 'review\tNN\n']
989860


In [165]:
with open('hmm_vocab.txt','r') as f:
  voc_l=f.read().split('\n')
print(voc_l[:50])
print(len(voc_l))

['!', '#', '$', '%', '&', "'", "''", "'40s", "'60s", "'70s", "'80s", "'86", "'90s", "'N", "'S", "'d", "'em", "'ll", "'m", "'n'", "'re", "'s", "'til", "'ve", '(', ')', ',', '-', '--', '--n--', '--unk--', '--unk_adj--', '--unk_adv--', '--unk_digit--', '--unk_noun--', '--unk_punct--', '--unk_upper--', '--unk_verb--', '.', '...', '0.01', '0.0108', '0.02', '0.03', '0.05', '0.1', '0.10', '0.12', '0.13', '0.15']
23777


In [166]:
vocab={}
for i,voc in enumerate(sorted(voc_l)):
  vocab[voc]=i
count=0
for k,v in vocab.items():
  print(f"{k}: {v}")
  count+=1
  if count>20:
    break
print(len(vocab))

: 0
!: 1
#: 2
$: 3
%: 4
&: 5
': 6
'': 7
'40s: 8
'60s: 9
'70s: 10
'80s: 11
'86: 12
'90s: 13
'N: 14
'S: 15
'd: 16
'em: 17
'll: 18
'm: 19
'n': 20
23777


In [167]:
with open('WSJ_24.pos','r') as f:
  test_data=f.readlines()
print(test_data[:10])
print(len(test_data))

['The\tDT\n', 'economy\tNN\n', "'s\tPOS\n", 'temperature\tNN\n', 'will\tMD\n', 'be\tVB\n', 'taken\tVBN\n', 'from\tIN\n', 'several\tJJ\n', 'vantage\tNN\n']
34199


In [168]:
_,prep=preprocess(vocab,'test.words')
print("the length of the preprocessed test corpus is:- ",len(prep))
prep[:10]

the length of the preprocessed test corpus is:-  34199


['The',
 'economy',
 "'s",
 'temperature',
 'will',
 'be',
 'taken',
 'from',
 'several',
 '--unk--']

In [169]:
def create_dictionaries(training_corpus,vocab,verbose=True):
  emission_counts=defaultdict(int)
  transition_counts=defaultdict(int)
  tag_counts=defaultdict(int)

  prev_tag='--s--'
  i=0
  for word_tag in training_corpus:
    i+=1
    if i%50000==0 and verbose:
      print(f'word count = {i}')
    word,tag=get_word_tag(word_tag,vocab)
    transition_counts[(prev_tag,tag)]+=1
    emission_counts[(tag,word)]+=1
    tag_counts[tag]+=1
    prev_tag=tag

  return emission_counts,transition_counts,tag_counts


In [170]:
emission_counts,transition_counts,tag_counts=create_dictionaries(training_data,vocab)

word count = 50000
word count = 100000
word count = 150000
word count = 200000
word count = 250000
word count = 300000
word count = 350000
word count = 400000
word count = 450000
word count = 500000
word count = 550000
word count = 600000
word count = 650000
word count = 700000
word count = 750000
word count = 800000
word count = 850000
word count = 900000
word count = 950000


In [171]:
states=sorted(tag_counts.keys())
print("length of the states= ",len(states))
print(states)

length of the states=  54
['#', '$', "''", '(', ')', ',', '--s--', '--unk--', '--unk_adj--', '--unk_adv--', '--unk_digit--', '--unk_noun--', '--unk_punct--', '--unk_upper--', '--unk_verb--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [172]:
print('transition examples')
for ex in list(transition_counts.items())[:3]:
  print(ex)
print()
print("emission examples")
for ex in list(emission_counts.items())[:3]:
  print(ex)
print()
print('tags examples')
for ex in list(tag_counts.items())[:3]:
  print(ex)

print()
print('ambiguous examples')
for emi,cnt in list(emission_counts.items()):
  if(emi[1]=='back'):
    print(emi,cnt)


transition examples
(('--s--', 'IN'), 5042)
(('IN', 'DT'), 32353)
(('DT', 'NNP'), 8721)

emission examples
(('IN', 'In'), 1735)
(('DT', 'an'), 3142)
(('NNP', 'Oct.'), 317)

tags examples
('IN', 98537)
('DT', 81840)
('NNP', 86463)

ambiguous examples
('RB', 'back') 304
('VB', 'back') 20
('RP', 'back') 84
('JJ', 'back') 25
('NN', 'back') 29
('VBP', 'back') 4


In [173]:
def predict_pos(prep,y,emission_counts,vocab,states):
  num_corr=0
  all_words=set(emission_counts.keys())
  total=len(y)
  for word,tup in zip(prep,y):
    y_tup=tup.split()
    if len(y_tup)==2:
      true_label=y_tup[1]
    else:
      continue

    count_final=0
    pos_final=''
    if word in vocab:
      for state in states:
        key=(state,word)
        if key in emission_counts.keys():
          count=emission_counts[key]
          if count_final<count:
            count_final=count
            pos_final=state

      if true_label==pos_final:
        num_corr+=1
  accuracy=num_corr/total
  return accuracy



In [174]:
accuracy_predict_pos=predict_pos(prep,test_data,emission_counts,vocab,states)
print("The accuracy is: ", accuracy_predict_pos)

The accuracy is:  0.8658147899061376


In [175]:
def create_transition_matrix(alpha,tag_counts,transition_counts):
  all_tags=sorted(tag_counts.keys())
  num_tags=len(all_tags)
  A=np.zeros((num_tags,num_tags))
  trans_keys=set(transition_counts.keys())
  for i in range(num_tags):
    for j in range(num_tags):
      count=0
      key=(all_tags[i],all_tags[j])
      if key in trans_keys:
        count=transition_counts[key]
      prev_tag_count=tag_counts[all_tags[i]]
      A[i,j]=(count+alpha)/(prev_tag_count + num_tags*alpha)

  return A






In [176]:
alpha=0.001
A= create_transition_matrix(alpha,tag_counts,transition_counts)
print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[3,1]:.4f}")

print("View a subset of transition matrix A")
A_sub = pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )
print(A_sub)


A at row 0, col 0: 0.000007040
A at row 3, col 1: 0.1691
View a subset of transition matrix A
          NNPS       NNS           PDT           POS       PRP
NNPS  0.014563  0.010922  4.045219e-07  2.548528e-02  0.001618
NNS   0.000035  0.010383  5.184771e-05  9.260388e-03  0.004682
PDT   0.000003  0.000003  2.702308e-06  2.702308e-06  0.000003
POS   0.003103  0.127111  1.149286e-07  1.149286e-07  0.000115
PRP   0.000057  0.000976  1.148281e-04  1.032993e-03  0.001607


In [177]:
def create_emission_matrix(alpha,tag_counts,emission_counts,vocab):
  all_tags=sorted(tag_counts.keys())
  len_vocab=len(vocab)
  len_tags=len(tag_counts)
  B=np.zeros((len_tags,len_vocab))
  emi_keys=set(emission_counts.keys())
  for i in range(len_tags):
    for j in range(len_vocab):
      count=0
      key=(all_tags[i],vocab[j])
      if key in emi_keys:
        count=emission_counts[key]
      pos_count=tag_counts[all_tags[i]]
      B[i,j]=(count+alpha)/(pos_count+alpha*len_vocab)

  return B


In [178]:
alpha = 0.001
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))

print(f"View Matrix position at row 0, column 0: {B[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {B[3,1]:.9f}")
cidx  = ['725','adroitly','engineers', 'promoted', 'synergy']
cols = [vocab[a] for a in cidx]
rvals =['CD','NN','NNS', 'VB','RB','RP']
rows = [states.index(a) for a in rvals]
B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns = cidx )
print(B_sub)

View Matrix position at row 0, column 0: 0.000006032
View Matrix position at row 3, column 1: 0.000000720
              725      adroitly     engineers      promoted       synergy
CD   8.913567e-05  2.970199e-08  2.970199e-08  2.970199e-08  2.970199e-08
NN   7.714813e-09  7.714813e-09  7.714813e-09  7.714813e-09  2.315215e-05
NNS  1.726973e-08  1.726973e-08  4.835698e-04  1.726973e-08  1.726973e-08
VB   3.847519e-08  3.847519e-08  3.847519e-08  3.847519e-08  3.847519e-08
RB   3.271740e-08  6.546752e-05  3.271740e-08  3.271740e-08  3.271740e-08
RP   3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07


In [179]:
def initialize(states,tag_counts,A,B,corpus,vocab):
  num_tags=len(set(tag_counts.keys()))
  best_prob=np.zeros((num_tags,len(corpus)))
  best_path=np.zeros((num_tags,len(corpus)),dtype=int)
  s_indx=states.index('--s--')
  for i in range(num_tags):
    if A[s_indx,i]==0:
      best_prob[i,0]=float("-inf")
    else:
      best_prob[i,0]=math.log(A[s_indx,i])+math.log(B[i,vocab[corpus[0]]])

  return best_prob,best_path





In [180]:
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)

In [181]:
print(f"best_probs[0,0]: {best_probs[0,0]:.4f}")
print(f"best_paths[2,3]: {best_paths[2,3]:.4f}")

best_probs[0,0]: -22.6098
best_paths[2,3]: 0.0000


In [182]:
def viterbi_forward(A,B,test_corpus,best_probs,best_paths,vocabs,verbose=True):
  num_tags=best_probs.shape[0]
  for i in range(1,len(test_corpus)):
    if i % 5000 == 0 and verbose:
      print("Words processed: {:>8}".format(i))

    for j in range(num_tags):
      best_prob_i=float("-inf")
      best_path_i=None

      for k in range(num_tags):
        prob = best_probs[k,i-1]+math.log(A[k,j])+math.log(B[j,vocabs[test_corpus[i]]])
        if prob>best_prob_i:
          best_prob_i=prob
          best_path_i=k

      best_probs[j,i] = best_prob_i
      best_paths[j,i] = best_path_i

  return best_probs,best_paths





In [186]:
#vocab['unk']

In [184]:
best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)

Words processed:     5000
Words processed:    10000
Words processed:    15000
Words processed:    20000
Words processed:    25000
Words processed:    30000


In [185]:
print(f"best_probs[0,1]: {best_probs[0,1]:.4f}")
print(f"best_probs[0,4]: {best_probs[0,4]:.4f}")

best_probs[0,1]: -24.7822
best_probs[0,4]: -49.5150


In [187]:
def viterbi_backward_pass(best_probs,best_paths,corpus,states):
  m = best_paths.shape[1]
  z = [None] * m
  num_tags = best_probs.shape[0]
  best_prob_for_last_word = float('-inf')
  pred = [None] * m
  for k in range(num_tags):
    if best_probs[k,m-1]>best_prob_for_last_word:
      best_prob_for_last_word = best_probs[k,m-1]
      z[m - 1] = k

  pred[m - 1] = states[z[m-1]]

  for i in range(m-1, 0, -1):
    pos_tag_for_word_i = z[i]
    z[i - 1] = best_paths[pos_tag_for_word_i,i]
    pred[i - 1] = states[z[i-1]]

  return pred

In [188]:
pred = viterbi_backward_pass(best_probs, best_paths, prep, states)
m=len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

The prediction for pred[-7:m-1] is: 
 ['see', 'them', 'here', 'with', 'us', '.'] 
 ['VB', 'PRP', 'RB', 'IN', 'PRP', '.'] 

The prediction for pred[0:8] is: 
 ['DT', 'NN', 'POS', 'NN', 'MD', 'VB', 'VBN'] 
 ['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken']


In [190]:
print('The third word is:', prep[3])
print('Your prediction is:', pred[3])
print('Your corresponding label y is: ', test_data[3])

The third word is: temperature
Your prediction is: NN
Your corresponding label y is:  temperature	NN



In [191]:
def compute_accuracy(pred,y):
  num_correct=0
  tot=0
  for prediction,y in zip(pred,y):
    word_tag_tuple=y.split()
    if len(word_tag_tuple)!=2:
      continue
    word,tag=word_tag_tuple
    if prediction==tag:
      num_correct+=1
    tot+=1

  accuracy=num_correct/tot
  return accuracy

In [193]:
print(f"Accuracy of the Viterbi algorithm is {compute_accuracy(pred, test_data):.4f}")

Accuracy of the Viterbi algorithm is 0.9289
